# Activities

Note: the practice happens almost exclusively on paper, potentially augmented by multiple choice questions in order to submit an answer that shows understanding.

The activities performed are:
- 1 - Build a single RESTful controller against an EF database
- 2 - Add pagination to `GetAll`
- 3 - Add very simple optional filtering to `GetAll` (`GreaterThan`)
- 4 - Add very simple optional sorting to `GetAll` (by a single property)
- 5 - Add generic predicate filtering to `GetAll`

### 5 Generic predicate filtering
To use generic predicates to select entities of our database we need use LINQ dynamically by means of `Expression Trees` [1]. Expression trees allow to build/compose dynamically LINQ objects that can be used to test for example whether to select or not an entity, to order the result of a query, etc.



In the following a schematized version of a simple application is presented. In the schema a client is trying to get all the actors from the remote database that are called `Pippo` and which ages are 18. To do so the client first builds a filter expression (**1**) that captures the just mentioned conditions, and then calls (**2**) the remote method `GetAll` to which the just built filter expression (_our predicate_) is passed as a json string. 



The controller captures the call request (**3**) and forwards it to the right method (**4**). The method `GetAll` converts the input filter expression to an equivalent version in `Expression Tree`. The converted expression and the rest of the query is then passed to EF (**5**) which takes will take care of selecting the right actors. Once selected, the actors are returned back to the client (**6**). The client, which meanwhile was waiting for a response (**2.1**) receives the actors and saves them for future usages (**7**).



<img src="_1.jpg">



In the following the code for the client and the server is provided and commented. Use the above image to follow the structure of the solution.

#### Client
For this solution we will use a client written in C# (you can skip this part and use directly postman; in this case make sure to build correctly the body of the predicate to send via http). 


Create a a console application `dotnet new console` and add the Newtonsoft package to it `dotnet add package Newtonsoft.Json`. Newtonsoft is needed to parse the predicate into a json structure to send to the server. In the end the code should look like the following:


<img src="_2.jpg">

The classes needed for the correct functioning of the code above are:

<img src="_3.jpg">


Before you run the program make sure that the server (which definition is provided in the following section) is running.

#### Server

To implement the server we need to make a _webapi_ project (`dotnet new webapi`). Once created, add to it the design package (`dotnet add package Microsoft.EntityFrameworkCore.Design`), and by hand add a reference to the EntityFrameworkCore in the projects' setting file (the one that ends in *.csproj).


In the end the project setting file should look like this:

<img src="_4.jpg">

Once created create a folder Models and add the models from the previous practicums. You need to change for this implementaiton the constructor of the context. It should eventually look like the following (in this example we use the 1.Movie-N.Actors example):


<img src="_5.jpg">

Modify the Startup.cs file to connect our model to the controller once we run our server.

<img src="_6.jpg">


In the _Controllers_ folder add a MovieController.cs file. In this file we will configure the controller, which will take care of providing our movies and actors to the client.

In the just made MovieController file add the folowing usings:

<img src="_7.jpg">

Copy the _Filter_ class from the clients' code. We need this class, because the deserializer in our server needs to know how to build-back the seriealized predicate. 


<img src="_8.jpg">

Add a MovieController class which contrcutor will take care of saving locally the database context, and filling the database with some initial values (for debugging).

<img src="_9.jpg">

Add now the controller that will return the actors that satisfy a certain predicate

<img src="_10.jpg">

To conclude in the following the method that converts our filter predicate to an expression tree is shown.


<img src="_11.jpg">

To run the server use the command `dotnet run`. Now test it by calling run on the client written above.


References:
- [1] https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/concepts/expression-trees/
